In [1]:
import torch
from gliner import GLiNER

In [12]:
# first load your model
model = GLiNER.from_pretrained("gliner-community/gliner_medium-v2.5")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

## Option 1

In [14]:
# save
model.save_pretrained("gliner_Med")

In [15]:
# load
loaded_model = GLiNER.from_pretrained("gliner_Med", load_tokenizer = True, local_files_only=True)

config.json not found in /media/wukong/dataext4/GLiNER/examples/gliner_Med


## Option 2

In [16]:
def save_model(current_model, path):
    config = current_model.config
    dict_save = {"model_weights": current_model.state_dict(), "config": config}
    torch.save(dict_save, path)


def load_model(path, model_name=None):
    dict_load = torch.load(path, map_location=torch.device('cpu'))
    config = dict_load["config"]
    print(f"'{config.model_name}' should be available for local processing")
    if model_name is not None:
        config.model_name = model_name
    loaded_model = GLiNER(config)
    loaded_model.load_state_dict(dict_load["model_weights"])
    return loaded_model

In [17]:
# save the model weight
save_model(model, "model_weight.pt")

In [18]:
# load model weight
loaded_model = load_model("model_weight.pt")
print("success !!")

/tmp/ipykernel_289509/2930957027.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dict_load = torch.load(path, map_location=torch.device('cpu'))


'microsoft/deberta-v3-base' should be available for local processing


RuntimeError: Error(s) in loading state_dict for GLiNER:
	size mismatch for model.token_rep_layer.bert_layer.model.embeddings.word_embeddings.weight: copying a param with shape torch.Size([128003, 768]) from checkpoint, the shape in current model is torch.Size([128100, 768]).

## Testing

In [19]:
text = """
Libretto by Marius Petipa, based on the 1822 novella ``Trilby, ou Le Lutin d'Argail`` by Charles Nodier, first presented by the Ballet of the Moscow Imperial Bolshoi Theatre on January 25/February 6 (Julian/Gregorian calendar dates), 1870, in Moscow with Polina Karpakova as Trilby and Ludiia Geiten as Miranda and restaged by Petipa for the Imperial Ballet at the Imperial Bolshoi Kamenny Theatre on January 17–29, 1871 in St. Petersburg with Adèle Grantzow as Trilby and Lev Ivanov as Count Leopold.
"""

labels = ["person", "book", "location", "date", "actor", "character"]

entities = loaded_model.predict_entities(text, labels, threshold=0.4)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Marius Petipa => person
1822 => date
Trilby, ou Le Lutin d'Argail => book
Charles Nodier => person
Moscow => location
January 25/February 6 => date
1870 => date
Moscow => location
Polina Karpakova => actor
Trilby => character
Ludiia Geiten => actor
Miranda => character
Petipa => person
Imperial Bolshoi Kamenny Theatre => location
January 17–29, 1871 => date
St. Petersburg => location
Adèle Grantzow => actor
Trilby => character
Lev Ivanov => actor
Count Leopold => character
